In [11]:
import sys
sys.path.append('..')

import random

import numpy as np
import pandas as pd
import torch
import transformers

from transformers import PatchTSTConfig, Trainer, TrainingArguments, EarlyStoppingCallback
from transformers import PatchTSTForPrediction as PatchTSTForPredictionOG
from TimeSeriesJEPA.models import PatchTSTModelJEPA, PatchTSTForPrediction
from TimeSeriesJEPA.datasets.benchmark_dataset import BenchmarkDataset

In [12]:
csv_path = r"D:\Coursework\MTS\dataset\ETT-small\ETTh2.csv"
dataset = "ETTh2"
num_workers = 4  # Reduce this if you have low number of CPU cores
batch_size = 32  # Reduce if not enough GPU memory available
context_length = 512
forecast_horizon = 96
patch_length = 8
num_input_channels=7

In [13]:

trainwindowds = BenchmarkDataset(csv_path=csv_path, context_length=context_length, prediction_length=forecast_horizon, flag='train', returndict=True)
valwindowds = BenchmarkDataset(csv_path=csv_path, context_length=context_length, prediction_length=forecast_horizon, flag='test', returndict=True)
print("dataset loaded, total size: ", len(trainwindowds), len(valwindowds))

Total data size:  (17420, 7)
Total data size:  (17420, 7)
dataset loaded, total size:  8033 2785


In [14]:
print("Loading prediction model")

config = PatchTSTConfig(
    do_mask_input=False,
    context_length=context_length,
    patch_length=patch_length,
    num_input_channels=num_input_channels,
    patch_stride=8,
    prediction_length=forecast_horizon,
    d_model=64,
    num_attention_heads=4,
    num_hidden_layers=3,
    ffn_dim=64,
    dropout=0.05,
    head_dropout=0.2,
    pooling_type=None,
    channel_attention=False,
    scaling="std",
    loss="mse",
    pre_norm=True,
    norm_type="batchnorm",
    positional_encoding_type = "sincos"
)

model = PatchTSTForPredictionOG(config=config)

Loading prediction model


In [15]:
model.cuda()

PatchTSTForPrediction(
  (model): PatchTSTModel(
    (scaler): PatchTSTScaler(
      (scaler): PatchTSTStdScaler()
    )
    (patchifier): PatchTSTPatchify()
    (masking): Identity()
    (encoder): PatchTSTEncoder(
      (embedder): PatchTSTEmbedding(
        (input_embedding): Linear(in_features=8, out_features=64, bias=True)
      )
      (positional_encoder): PatchTSTPositionalEncoding(
        (positional_dropout): Identity()
      )
      (layers): ModuleList(
        (0-2): 3 x PatchTSTEncoderLayer(
          (self_attn): PatchTSTAttention(
            (k_proj): Linear(in_features=64, out_features=64, bias=True)
            (v_proj): Linear(in_features=64, out_features=64, bias=True)
            (q_proj): Linear(in_features=64, out_features=64, bias=True)
            (out_proj): Linear(in_features=64, out_features=64, bias=True)
          )
          (dropout_path1): Identity()
          (norm_sublayer1): PatchTSTBatchNorm(
            (batchnorm): BatchNorm1d(64, eps=1e-05, mom

In [16]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model.parameters()])
print("encoder parameters: ", params)

encoder parameters:  473632


In [17]:
train_args = TrainingArguments(
    output_dir=r"checkpoints\finetuned_og",
    overwrite_output_dir=True,
    learning_rate=0.0001,
    num_train_epochs=30,
    do_eval=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    dataloader_num_workers=1,  # num_workers,
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=3,
    logging_dir=r"checkpoints\finetuned_og\logs",
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
    greater_is_better=False,  # For loss
    label_names=["future_values"],
)

# Create a new early stopping callback with faster convergence properties
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=15,  # Number of epochs with no improvement after which to stop
    early_stopping_threshold=0.001,  # Minimum improvement required to consider as improvement
)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=trainwindowds,
    eval_dataset=valwindowds,
    callbacks=[early_stopping_callback],
)

c:\Users\Omen\anaconda3\envs\mts\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
print("\n\nDoing forecasting training")
trainer.train()



Doing forecasting training


  0%|          | 0/7560 [00:00<?, ?it/s]

{'loss': 0.4713, 'grad_norm': 0.5749430656433105, 'learning_rate': 9.666666666666667e-05, 'epoch': 1.0}


  0%|          | 0/88 [00:00<?, ?it/s]

{'eval_loss': 0.2816503643989563, 'eval_runtime': 5.5824, 'eval_samples_per_second': 498.889, 'eval_steps_per_second': 15.764, 'epoch': 1.0}
{'loss': 0.4468, 'grad_norm': 32.79212951660156, 'learning_rate': 9.333333333333334e-05, 'epoch': 2.0}


...

  0%|          | 0/88 [00:00<?, ?it/s]

{'eval_loss': 0.3022763729095459, 'eval_runtime': 5.5061, 'eval_samples_per_second': 505.806, 'eval_steps_per_second': 15.982, 'epoch': 17.0}
{'loss': 0.2486, 'grad_norm': 4.148008823394775, 'learning_rate': 4e-05, 'epoch': 18.0}


  0%|          | 0/88 [00:00<?, ?it/s]

{'eval_loss': 0.3059213161468506, 'eval_runtime': 5.6237, 'eval_samples_per_second': 495.224, 'eval_steps_per_second': 15.648, 'epoch': 18.0}
{'train_runtime': 288.3963, 'train_samples_per_second': 835.621, 'train_steps_per_second': 26.214, 'train_loss': 0.30554847448170713, 'epoch': 18.0}


TrainOutput(global_step=4536, training_loss=0.30554847448170713, metrics={'train_runtime': 288.3963, 'train_samples_per_second': 835.621, 'train_steps_per_second': 26.214, 'total_flos': 1472687363653632.0, 'train_loss': 0.30554847448170713, 'epoch': 18.0})

In [19]:
trainer.evaluate(valwindowds)

  0%|          | 0/88 [00:00<?, ?it/s]

{'eval_loss': 0.27863338589668274,
 'eval_runtime': 5.3796,
 'eval_samples_per_second': 517.695,
 'eval_steps_per_second': 16.358,
 'epoch': 18.0}

In [4]:
print("Loading pretrained encoder model")
encoder_model = PatchTSTModelJEPA.from_pretrained(r"D:\Coursework\MTS\timeseriesJEPA\results\PatchTST_etth2_sl512_enc_dm64_nh4_el3_fd64_pred_dm32_nh2_el1_fd32_bs256_lr0.0001_pe50_data_nenc1_npred4_clean_data\checkpoint-1600")
print("Done")
encoder_model.cuda()

Loading pretrained encoder model
Done


PatchTSTModelJEPA(
  (scaler): PatchTSTScaler(
    (scaler): PatchTSTStdScaler()
  )
  (patchifier): PatchTSTPatchify()
  (encoder): PatchTSTEncoder(
    (embedder): PatchTSTEmbedding(
      (input_embedding): Linear(in_features=8, out_features=64, bias=True)
    )
    (positional_encoder): PatchTSTPositionalEncoding(
      (positional_dropout): Identity()
    )
    (layers): ModuleList(
      (0-2): 3 x PatchTSTEncoderLayer(
        (self_attn): PatchTSTAttention(
          (k_proj): Linear(in_features=64, out_features=64, bias=True)
          (v_proj): Linear(in_features=64, out_features=64, bias=True)
          (q_proj): Linear(in_features=64, out_features=64, bias=True)
          (out_proj): Linear(in_features=64, out_features=64, bias=True)
        )
        (dropout_path1): Identity()
        (norm_sublayer1): PatchTSTBatchNorm(
          (batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (ff): Sequential(
          (0):

In [5]:
print("Loading prediction model")

config = PatchTSTConfig(
    do_mask_input=False,
    context_length=context_length,
    patch_length=patch_length,
    num_input_channels=num_input_channels,
    patch_stride=patch_length,
    prediction_length=forecast_horizon,
    d_model=64,
    num_attention_heads=4,
    # num_hidden_layers=4,
    ffn_dim=128,
    dropout=0.05,
    head_dropout=0.2,
    pooling_type=None,
    channel_attention=False,
    scaling="std",
    loss="mse",
    pre_norm=True,
    norm_type="batchnorm",
    positional_encoding_type = "sincos"
)

model = PatchTSTForPrediction(config=config, encoder_model=encoder_model)
model.cuda()

Loading prediction model


PatchTSTForPrediction(
  (model): PatchTSTModelJEPA(
    (scaler): PatchTSTScaler(
      (scaler): PatchTSTStdScaler()
    )
    (patchifier): PatchTSTPatchify()
    (encoder): PatchTSTEncoder(
      (embedder): PatchTSTEmbedding(
        (input_embedding): Linear(in_features=8, out_features=64, bias=True)
      )
      (positional_encoder): PatchTSTPositionalEncoding(
        (positional_dropout): Identity()
      )
      (layers): ModuleList(
        (0-2): 3 x PatchTSTEncoderLayer(
          (self_attn): PatchTSTAttention(
            (k_proj): Linear(in_features=64, out_features=64, bias=True)
            (v_proj): Linear(in_features=64, out_features=64, bias=True)
            (q_proj): Linear(in_features=64, out_features=64, bias=True)
            (out_proj): Linear(in_features=64, out_features=64, bias=True)
          )
          (dropout_path1): Identity()
          (norm_sublayer1): PatchTSTBatchNorm(
            (batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True

In [6]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print("encoder parameters: ", params)

encoder parameters:  393312


In [7]:
params = sum([np.prod(p.size()) for p in encoder_model.parameters()])
print("encoder parameters: ", params)

encoder parameters:  80448


In [8]:
train_args = TrainingArguments(
    output_dir=r"checkpoints\finetuned",
    overwrite_output_dir=True,
    learning_rate=0.0001,
    num_train_epochs=30,
    do_eval=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    dataloader_num_workers=1,  # num_workers,
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=3,
    logging_dir=r"checkpoints\finetuned\logs",
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
    greater_is_better=False,  # For loss
    label_names=["future_values"],
)

# Create a new early stopping callback with faster convergence properties
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=5,  # Number of epochs with no improvement after which to stop
    early_stopping_threshold=0.001,  # Minimum improvement required to consider as improvement
)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=trainwindowds,
    eval_dataset=valwindowds,
    callbacks=[early_stopping_callback],
)

c:\Users\Omen\anaconda3\envs\mts\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
print("\n\nDoing forecasting training")
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.




Doing forecasting training


wandb: Currently logged in as: vg2523 (hpml_4). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


  0%|          | 0/7560 [00:00<?, ?it/s]

{'loss': 0.7263, 'grad_norm': 2.5108461380004883, 'learning_rate': 9.666666666666667e-05, 'epoch': 1.0}


...

  0%|          | 0/88 [00:00<?, ?it/s]

{'eval_loss': 0.2783200144767761, 'eval_runtime': 6.3509, 'eval_samples_per_second': 438.521, 'eval_steps_per_second': 13.856, 'epoch': 25.0}
{'loss': 0.463, 'grad_norm': 2.5362839698791504, 'learning_rate': 1.3333333333333333e-05, 'epoch': 26.0}


  0%|          | 0/88 [00:00<?, ?it/s]

{'eval_loss': 0.27611565589904785, 'eval_runtime': 6.138, 'eval_samples_per_second': 453.734, 'eval_steps_per_second': 14.337, 'epoch': 26.0}
{'train_runtime': 379.1393, 'train_samples_per_second': 635.624, 'train_steps_per_second': 19.94, 'train_loss': 0.5231814401927012, 'epoch': 26.0}


TrainOutput(global_step=6552, training_loss=0.5231814401927012, metrics={'train_runtime': 379.1393, 'train_samples_per_second': 635.624, 'train_steps_per_second': 19.94, 'total_flos': 2127789964984320.0, 'train_loss': 0.5231814401927012, 'epoch': 26.0})

In [10]:
trainer.evaluate(valwindowds)

  0%|          | 0/88 [00:00<?, ?it/s]

{'eval_loss': 0.27611565589904785,
 'eval_runtime': 5.4027,
 'eval_samples_per_second': 515.485,
 'eval_steps_per_second': 16.288,
 'epoch': 26.0}